In [126]:
import pandas as pd
from geopy.distance import geodesic
import math
import getweatherdata
import os
from IGC_file_parse import getfile as igc
import folium as fm


def calculate_bearing(row):
    lat1 = math.radians(row["previous_latitude"])
    lon1 = math.radians(row["previous_longitude"])
    lat2 = math.radians(row["latitude"])
    lon2 = math.radians(row["longitude"])

    delta_lon = lon2 - lon1

    x = math.sin(delta_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
                                           * math.cos(lat2) * math.cos(delta_lon))

    initial_bearing = math.atan2(x, y)
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = int((initial_bearing + 360) % 360)

    return compass_bearing


def calculate_distance(row):
    return geodesic((row["previous_latitude"], row["previous_longitude"]), (row["latitude"], row["longitude"])).meters


def get_weather_data(row):
    lat = float(row["latitude"])
    lon = float(row["longitude"])
    timestamp = row["datetime"]
    filename = row["filename"]
    result = getweatherdata.main(filename, lat, lon, timestamp)
    return result


df = igc("flightlogs\\2024-05-19 10_05_00.igc", 1)
# df = igc("flightlogs\\2018-07-16 09_13_56.igc", 1)

df.head()
df.info()
df.head()


190524
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4966 entries, 0 to 4965
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   filename             4966 non-null   object        
 1   datetime             4966 non-null   datetime64[ns]
 2   pilot                4966 non-null   object        
 3   latitude             4966 non-null   float64       
 4   longitude            4966 non-null   float64       
 5   gps_altitude_m       4966 non-null   int64         
 6   pressure_altitude_m  4966 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 271.7+ KB


,filename,datetime,pilot,latitude,longitude,gps_altitude_m,pressure_altitude_m
0,2024-05-19 10_05_00.igc,2024-05-19 10:05:00,Serkan Kurd,40.0312,32.3283,1090,1144
1,2024-05-19 10_05_00.igc,2024-05-19 10:05:01,Serkan Kurd,40.0312,32.3283,1090,1144
2,2024-05-19 10_05_00.igc,2024-05-19 10:05:02,Serkan Kurd,40.0312,32.3283,1090,1144
3,2024-05-19 10_05_00.igc,2024-05-19 10:05:03,Serkan Kurd,40.0312,32.3283,1090,1144
4,2024-05-19 10_05_00.igc,2024-05-19 10:05:04,Serkan Kurd,40.0312,32.3283,1090,1144


In [127]:
df["previous_latitude"] = df["latitude"].shift(1)
df["previous_longitude"] = df["longitude"].shift(1)
df["distance_from_takeoff_m"] = df.apply(
    lambda row: geodesic((row["latitude"], row["longitude"]), (df["latitude"].iloc[0], df["longitude"].iloc[0])).meters, axis=1)
df.drop(df.index[0], inplace=True)

In [128]:
df["distance_m"] = df.apply(lambda row: geodesic((row["previous_latitude"], row["previous_longitude"]), (row["latitude"], row["longitude"])).meters, axis=1)
df["speed_km/s"] = ((df["distance_m"]/1000) /
                    (df["datetime"].diff().dt.total_seconds()/3600))
df["climb_m"] = df["gps_altitude_m"].diff()
df["climb_m(10sec)"] = df["gps_altitude_m"].diff(10)
df["climb_rate_m/s"] = df["climb_m"] / df["datetime"].diff().dt.total_seconds()
df["bearing"] = df.apply(calculate_bearing, axis=1)
df["delta_bearing"] = abs((df["bearing"].diff() + 180) % 360 - 180)
df["delta_bearing(6sec)"] = df["delta_bearing"].shift(1) + df["delta_bearing"].shift(2) + df["delta_bearing"].shift(3) + df["delta_bearing"].shift(4) + df["delta_bearing"].shift(5) + df["delta_bearing"].shift(6)
df["glide_ratio"] = df.apply(
    lambda row: row["distance_m"] /
    abs(row["climb_m"]) if row["climb_m"] != 0 else 0,
    axis=1)

df.fillna(0, inplace=True)
df.head()

,filename,datetime,pilot,latitude,longitude,gps_altitude_m,pressure_altitude_m,previous_latitude,previous_longitude,distance_from_takeoff_m,distance_m,speed_km/s,climb_m,climb_m(10sec),climb_rate_m/s,bearing,delta_bearing,delta_bearing(6sec),glide_ratio
1,2024-05-19 10_05_00.igc,2024-05-19 10:05:01,Serkan Kurd,40.0312,32.3283,1090,1144,40.0312,32.3283,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
2,2024-05-19 10_05_00.igc,2024-05-19 10:05:02,Serkan Kurd,40.0312,32.3283,1090,1144,40.0312,32.3283,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
3,2024-05-19 10_05_00.igc,2024-05-19 10:05:03,Serkan Kurd,40.0312,32.3283,1090,1144,40.0312,32.3283,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
4,2024-05-19 10_05_00.igc,2024-05-19 10:05:04,Serkan Kurd,40.0312,32.3283,1090,1144,40.0312,32.3283,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
5,2024-05-19 10_05_00.igc,2024-05-19 10:05:05,Serkan Kurd,40.0312,32.3283,1089,1144,40.0312,32.3283,0.0,0.0,0.0,-1.0,0.0,-1.0,0,0.0,0.0,0.0


In [129]:
# find and delete before take off
for index, row in df.iterrows():
    if row["speed_km/s"] > 10:
        print(index)
        df = df.iloc[index:]
        df.reset_index(drop=True, inplace=True)
        break
df["elapsed_time"] = (df["datetime"] - df["datetime"].iloc[0]).dt.total_seconds()

df["zone"] = df.apply(lambda row: "thermal" if row["delta_bearing(6sec)"] > 45 or row["climb_m(10sec)"] > 0 else "standart", axis=1)
# df[["temp", "pressure", "humidity", "dew_point", "wind_speed", "wind_deg"]] = df.apply(get_weather_data, axis=1, result_type="expand")
df.drop(["filename", "pilot", "previous_latitude", "previous_longitude",
        "climb_m", "distance_m"], axis=1, inplace=True)
df.to_csv(os.path.join("data", "flight_data_processed.csv"), index=False)
df.head()     

16


,datetime,latitude,longitude,gps_altitude_m,pressure_altitude_m,distance_from_takeoff_m,speed_km/s,climb_m(10sec),climb_rate_m/s,bearing,delta_bearing,delta_bearing(6sec),glide_ratio,elapsed_time,zone
0,2024-05-19 10:05:17,40.031233,32.328217,1086,1142,8.018243,16.746131,-3.0,-3.0,293,10.0,221.0,1.550568,0.0,thermal
1,2024-05-19 10:05:18,40.031250,32.328150,1084,1139,13.955109,21.541274,-5.0,-2.0,288,5.0,141.0,2.991844,1.0,thermal
2,2024-05-19 10:05:19,40.031233,32.328083,1082,1138,18.860302,21.541274,-7.0,-2.0,251,37.0,146.0,2.991844,2.0,thermal
3,2024-05-19 10:05:20,40.031250,32.328017,1083,1136,24.812964,21.541274,-6.0,1.0,288,37.0,183.0,5.983687,3.0,thermal
4,2024-05-19 10:05:21,40.031300,32.327983,1084,1133,29.220851,22.458060,-5.0,1.0,332,44.0,183.0,6.238350,4.0,thermal


In [ ]:
# coordinates = df[["latitude", "longitude"]].values.tolist()
# zone = df["zone"].values.tolist()
# map = fm.Map(location=coordinates[0], zoom_start=13)
# for x in range(len(coordinates)):
#     if zone[x] == "thermal":
#         color_ = 'red'
#     else:
#         color_ = 'blue'
#     fm.CircleMarker(location=coordinates[x], radius=1, color=color_,
#                     fill=True, fill_color=color_, popup=zone).add_to(map)
# fm.Marker(location=coordinates[0], popup="Takeoff").add_to(map)
# map.save("map.html")